https://colab.research.google.com/drive/1hc8G2WY_4P_0Tri-lZ0HmVDdX6MKy5LV#scrollTo=rcFWDyYIt733

In [ ]:
# Remove all potentially conflicting packages
!pip uninstall -y transformers diffusers huggingface-hub sentence-transformers accelerate peft

!pip install -q torch torchvision
!pip install -q huggingface-hub==0.19.4
!pip install -q accelerate==0.25.0
!pip install -q transformers==4.36.2
!pip install -q diffusers==0.24.0
!pip install -q safetensors==0.4.1
!pip install -q peft==0.7.1
!pip install -q flask-cors pyngrok flask

from google.colab import userdata
NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# Configure ngrok
!ngrok authtoken $NGROK_TOKEN

print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import torch
from diffusers import DiffusionPipeline
import base64
import io
from PIL import Image

!pkill ngrok

def initialize_pipeline():
    try:
        pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/sdxl-turbo",
        torch_dtype=torch.float16,
        variant="fp16",
        )
        if torch.cuda.is_available():
            pipe = pipe.to("cuda")
        return pipe
    except Exception as e:
        print(f"Error initializing pipeline: {str(e)}")
        return None

app = Flask(__name__)
CORS(app)

# Initialize the pipeline
pipe = initialize_pipeline()

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'gpu_available': torch.cuda.is_available(),
        'torch_version': torch.__version__,
        'cuda_version': torch.version.cuda if torch.cuda.is_available() else None
    })

@app.route('/generate_frame', methods=['POST'])
def generate_frame():
    if pipe is None:
        return jsonify({'success': False, 'error': 'Pipeline not initialized'}), 500

    try:
        data = request.json
        prompt = data.get('prompt', '')

        if not prompt:
            return jsonify({'success': False, 'error': 'No prompt provided'}), 400

        # Optional parameters with defaults
        height = data.get('height', 512)
        width = data.get('width', 512)
        num_inference_steps = data.get('num_inference_steps', 1)
        guidance_scale = data.get('guidance_scale', 0.0)

        # Generate image
        image = pipe(
            prompt=prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            height=height,
            width=width
        ).images[0]

        # Convert to base64
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG", quality=95)
        img_str = base64.b64encode(buffered.getvalue()).decode()

        return jsonify({
            'success': True,
            'image': f'data:image/jpeg;base64,{img_str}'
        })

    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

# Setup ngrok and start server
try:
    public_url = ngrok.connect(5000)
    print(f'\nServer running at: {public_url}')
    print(f'Frame generation endpoint: {public_url}/generate_frame')
    print(f'Health check endpoint: {public_url}/health\n')

    # Start Flask server
    app.run(port=5000)
except Exception as e:
    print(f"Error starting server: {str(e)}")

In [ ]:
# Installation
"""
pip install flask flask-cors pyngrok replicate
"""

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import replicate
import base64
import requests
import io
from PIL import Image

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'healthy'})

@app.route('/generate_frames', methods=['POST'])
def generate_frames():
    try:
        data = request.json
        prompt = data.get('prompt', '')

        if not prompt:
            return jsonify({'success': False, 'error': 'No prompt provided'}), 400

        # Using AnimateDiff through Replicate API
        output = replicate.run(
            "lucataco/animate-diff:1531004ee4c98894ab11f8a4ce6206099e732c1da15121987a8eef54828f0663",
            input={
                "prompt": prompt,
                "negative_prompt": "blurry, bad quality, distorted",
                "num_frames": 16,  # Number of frames to generate
                "fps": 8,  # Frames per second
                "num_inference_steps": 20,
                "guidance_scale": 7.5,
            }
        )

        # Output will be a URL to the generated animation

        # Download and convert frames if needed
        if isinstance(output, list) and len(output) > 0:
            animation_url = output[0]
            response = requests.get(animation_url)

            if response.status_code == 200:
                # Convert to base64 for sending back
                img_str = base64.b64encode(response.content).decode()

                return jsonify({
                    'success': True,
                    'animation': f'data:image/gif;base64,{img_str}',
                    'url': animation_url
                })

        return jsonify({
            'success': False,
            'error': 'Failed to generate animation'
        }), 500

    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

# Setup test route to generate a sample animation
@app.route('/test', methods=['GET'])
def test_generation():
    test_prompt = "A cute cat walking in a garden, Studio Ghibli style"
    return generate_frames(), {
        'prompt': test_prompt
    }

def start_server():
    try:
        # Setup ngrok
        public_url = ngrok.connect(5000)
        print(f'\nServer running at: {public_url}')
        print(f'Frame generation endpoint: {public_url}/generate_frames')
        print(f'Test endpoint: {public_url}/test')
        print(f'Health check endpoint: {public_url}/health\n')

        # Start Flask server
        app.run(port=5000)
    except Exception as e:
        print(f"Error starting server: {str(e)}")

if __name__ == '__main__':
    start_server()